In [1]:
from datasets import load_dataset
import pandas as pd

subjqa = load_dataset("subjqa", name="electronics")
dfs = {split: dset.to_pandas() for split, dset in subjqa.flatten().items()}

for split, df in dfs.items():
    print(f"Number of questions in {split}: {df['id'].nunique()}")

Number of questions in train: 1295
Number of questions in test: 358
Number of questions in validation: 255


### Initializing a document store Using Docker

In [2]:
# from haystack.utils import launch_es

# launch_es()

In [3]:
# !curl -X GET "localhost:9200/?pretty"

In [4]:
!conda create -n haystack python=3.7
!conda activate haystack
!pip install farm-haystack

zsh:1: command not found: conda
zsh:1: command not found: conda


In [5]:
!docker pull docker.elastic.co/elasticsearch/elasticsearch:7.9.2
!docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

7.9.2: Pulling from elasticsearch/elasticsearch
Digest: sha256:2be3302537236874fdeca184c78a49aed17d5aca0f8fc3f6192a80e93e817cb4
Status: Image is up to date for docker.elastic.co/elasticsearch/elasticsearch:7.9.2
docker.elastic.co/elasticsearch/elasticsearch:7.9.2

What's next:
    View a summary of image vulnerabilities and recommendations → docker scout quickview docker.elastic.co/elasticsearch/elasticsearch:7.9.2
d9544cd91c0539dc3b6282499b340f37173da6e8831aa1dd70bc02168c3af561
docker: Error response from daemon: driver failed programming external connectivity on endpoint objective_hertz (7be12c8f4727fb4676bacb6efcdd431a3440bdddefd2444df23f2874643d47f7): Bind for 0.0.0.0:9200 failed: port is already allocated.


In [6]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

# return the document embedding for later use with dense retriever 
document_store = ElasticsearchDocumentStore(return_embedding=True)

/Users/macbook/Downloads/jupyter_env/lib/python3.12/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [7]:
for split, df in dfs.items():
    docs = [
        {"content": row["context"], "id": row["review_id"],
        "meta":{"item_id": row["title"], "question_id": row["id"], "split": split}} 
        for _,row in df.drop_duplicates(subset="context").iterrows()
    ]
    document_store.write_documents(documents=docs, index="document")
    
print(f"Loaded {document_store.get_document_count()} documents")

Loaded 1615 documents


### Initializing a Retriever

In [8]:
from haystack.retriever.sparse import ElasticSearchRetriever

es_retriever = ElasticSearchRetriever(document_store=document_store)

ModuleNotFoundError: No module named 'haystack.retriever'